Calculate taus:

on unit level:
1. Pearsonr trial average
2. STTC trial average
3. STTC trial concat
   
on trial level (not calculated yet):
1. Pearsonr per trial
2. ACF proper per trial
3. iSTTC per trial

In [1]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
empty_suffix = 'no'
data_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'

### Functions

In [6]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    """
    Calculate tau values per unit using single exponential fitting.

    Parameters:
    - acf_df (pd.DataFrame): DataFrame containing autocorrelation function (ACF) values.
    - acf_cols (list): List of column names containing ACF data.
    - start_idx (int): Starting index for fitting.

    Returns:
    - pd.DataFrame: DataFrame with fitting parameters and tau values per unit.
    """
    acf_2d = acf_df_[acf_cols_].values.astype(np.float64)
    n_units = acf_2d.shape[0]
    print(f'Calculating taus for {acf_2d.shape}')
    
    results = [fit_single_exp(acf_2d[i, :], start_idx_, func_single_exp_monkey) for i in range(n_units)]

    fit_popt_a, fit_popt_b, fit_popt_c, fit_tau, fit_r_squared, fit_log_message = zip(*[
        (popt[0] if isinstance(popt, np.ndarray) else np.nan,
         popt[1] if isinstance(popt, np.ndarray) else np.nan,
         popt[2] if isinstance(popt, np.ndarray) else np.nan,
         tau, r_squared, log_msg)
        for popt, _, tau, r_squared, log_msg in results
    ])

    tau_df = pd.DataFrame({
        'unit_id': acf_df_['unit_id'].values,
        'fit_a': fit_popt_a,
        'fit_b': fit_popt_b,
        'fit_c': fit_popt_c,
        'tau': fit_tau,
        'r_squared': fit_r_squared,
        'log_message': fit_log_message
    })

    float_cols = ['fit_a', 'fit_b', 'fit_c', 'tau', 'r_squared']
    tau_df[float_cols] = tau_df[float_cols].astype(float)
    
    return tau_df

### Calculate tau

#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [4]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [7]:
area = 'pfdl' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

Calculating taus for (539, 20)
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf 

In [11]:
tau_pearsonr_trial_avg_df

,unit_id,fit_a,fit_b,fit_c,tau,r_squared,log_message,tau_ms
0,0,0.719042,0.051840,-0.347076,19.290002,0.911963,ok,964.500088
1,1,NaN,NaN,NaN,NaN,NaN,ValueError,NaN
2,2,0.110746,1.005075,0.019225,0.994951,0.018581,ok,49.747526
3,3,0.279456,0.164319,0.663300,6.085726,0.902339,ok,304.286291
4,4,0.194175,0.277495,-0.088259,3.603664,0.686497,ok,180.183206
...,...,...,...,...,...,...,...,...
534,539,0.200108,0.138558,0.111026,7.217215,0.459831,ok,360.860736
535,540,0.475262,0.478004,-0.143530,2.092032,0.834083,ok,104.601593
536,541,0.809791,0.465480,-0.022679,2.148320,0.935643,ok,107.415989
537,542,0.277750,0.337468,-0.157115,2.963246,0.653134,ok,148.162323


In [16]:
tau_sttc_trial_concat_df.query('r_squared < 0')

,unit_id,fit_a,fit_b,fit_c,tau,r_squared,log_message,tau_ms
11,11,0.245144,17.262546,0.745102,0.057929,-6.817834e-09,ok,2.896444
16,16,-0.027214,17.276665,0.748771,0.057882,-2.638583e-09,ok,2.894077
20,20,-0.016710,16.262993,1.117895,0.061489,-5.471703e-09,ok,3.074465
35,35,-0.045797,10.982746,0.805402,0.091052,-1.361375e-08,ok,4.552595
54,54,0.085981,15.199560,1.133391,0.065791,-1.429648e-08,ok,3.289569
...,...,...,...,...,...,...,...,...
484,489,-0.009598,19.899459,1.118962,0.050253,-1.235758e-10,ok,2.512631
488,493,-0.012630,14.785423,5.944340,0.067634,-1.484684e-08,ok,3.381709
498,503,-0.049089,20.914123,1.180155,0.047815,-1.208277e-09,ok,2.390729
512,517,0.027741,20.252109,0.770251,0.049378,-1.676055e-10,ok,2.468879


In [ ]:
area = 'pfp' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

In [ ]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

In [ ]:
area = 'pfdl' 

# pearsonr 
# pearsonr_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(pearsonr_per_trial_df)}, dropping before tau calc ...')
# pearsonr_per_trial_df.dropna(inplace=True)
# pearsonr_per_trial_df.reset_index(drop=True, inplace=True)
# tau_pearsonr_per_trial_df = calc_tau_per_unit(pearsonr_per_trial_df, acf_cols[:-1], start_idx_=1)
# tau_pearsonr_per_trial_df['tau_ms'] = tau_pearsonr_per_trial_df['tau'] * bin_size
# tau_pearsonr_per_trial_df.insert(1, 'trial_id', pearsonr_per_trial_df['trial_id'])
# tau_pearsonr_per_trial_df.insert(2, 'condition_id', pearsonr_per_trial_df['condition_id'])
# tau_pearsonr_per_trial_df.insert(3, 'spike_count', pearsonr_per_trial_df['spike_count'])
# tau_pearsonr_per_trial_df.insert(4, 'fr_hz', pearsonr_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_pearsonr_per_trial_df)}')
# tau_pearsonr_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # acf formula
# pcf_proper_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pcf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(pcf_proper_per_trial_df)}, dropping before tau calc ...')
# pcf_proper_per_trial_df.dropna(inplace=True)
# pcf_proper_per_trial_df.reset_index(drop=True, inplace=True)
# tau_acf_proper_per_trial_df = calc_tau_per_unit(pcf_proper_per_trial_df, acf_cols, start_idx_=1)
# tau_acf_proper_per_trial_df['tau_ms'] = tau_acf_proper_per_trial_df['tau'] * bin_size
# tau_acf_proper_per_trial_df.insert(1, 'trial_id', pcf_proper_per_trial_df['trial_id'])
# tau_acf_proper_per_trial_df.insert(2, 'condition_id', pcf_proper_per_trial_df['condition_id'])
# tau_acf_proper_per_trial_df.insert(3, 'spike_count', pcf_proper_per_trial_df['spike_count'])
# tau_acf_proper_per_trial_df.insert(4, 'fr_hz', pcf_proper_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_acf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_acf_proper_per_trial_df)}')
# tau_acf_proper_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# isttc
sttc_per_trial_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
print(f'N rows with NaN {sttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(sttc_per_trial_df)}, dropping before tau calc ...')
sttc_per_trial_df.dropna(inplace=True)
sttc_per_trial_df.reset_index(drop=True, inplace=True)
tau_isttc_per_trial_df = calc_tau_per_unit(sttc_per_trial_df, acf_cols, start_idx_=1)
tau_isttc_per_trial_df['tau_ms'] = tau_isttc_per_trial_df['tau'] * bin_size
tau_isttc_per_trial_df.insert(1, 'trial_id', sttc_per_trial_df['trial_id'])
tau_isttc_per_trial_df.insert(2, 'condition_id', sttc_per_trial_df['condition_id'])
tau_isttc_per_trial_df.insert(3, 'spike_count', sttc_per_trial_df['spike_count'])
tau_isttc_per_trial_df.insert(4, 'fr_hz', sttc_per_trial_df['fr_hz'])
print(f'N rows with NaN {tau_isttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_isttc_per_trial_df)}')
tau_isttc_per_trial_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

In [ ]:
tau_acf_proper_per_trial_df

In [ ]:
pearsonr_per_trial_df

### todo (metrics)

In [ ]:
n_rows_with_nans = pearsonr_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(pearsonr_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_pearsonr_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_pearsonr_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

In [ ]:
n_rows_with_nans = sttc_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(sttc_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_sttc_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_sttc_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))